<p align="center">
<img src="Images/sorbonne_logo.png" alt="Logo" width="300"/>
</p>

# **Module 1 - Data Extraction & Manipulation**

* **Author**: Elia Landini
* **Student ID**: 12310239
* **Course**: EESM2-Financial Economics 
* **Supervisor**: XXXXXXXXXXXXXXXX
* **Reference Repository**: https://github.com/EliaLand/PVAR_japan_endogenous_money

### **1) PREFACE**

abcd

### **2) REQUIREMENTS SET-UP**

In [158]:
# Requirements.txt file installation
# !pip install -r requirements.txt

In [159]:
# Libraries import
import warnings
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import levene
from scipy.stats import ks_2samp
from scipy.stats import kstest
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.stattools import adfuller
import sklearn.tree
import sklearn.metrics
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing 
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, roc_curve, confusion_matrix,
                             precision_score, recall_score, f1_score,
                             accuracy_score, precision_recall_curve, auc, 
                             RocCurveDisplay, ConfusionMatrixDisplay)
from sklearn.linear_model import (LinearRegression, LogisticRegression)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight
import plotly.express as px
import openpyxl as pxl
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
from IPython.display import Image
import itertools
from arch.unitroot import PhillipsPerron

### **3) HELPER FUNCTIONS & GENERAL VARIABLES**

In [160]:
# Custom modules import
from Modules.FRED_module import fetch_FRED
from Modules.EUROSTAT_module import fetch_EUROSTAT
from Modules.WB_module import fetch_WB 
from Modules.YFINANCE_module import fetch_YFINANCE
from Modules.DBNOMICS_module import fetch_DBNOMICS

In [161]:
# Statistical Significance labelling 
def significance_stars(p):
    if p < 0.001:
        return "***"  
    elif p < 0.01:
        return "**"    
    elif p < 0.05:
        return "*"   
    else:
        return ""

In [162]:
# We supress potential warnings with this command
warnings.filterwarnings("ignore")

### **4) DATA RETRIEVAL**

##### **4.1) REAL EFFECTIVE EXCHANGE RATES**

In [163]:
# Real Effective Exchange Rates: CPI Based for Japan (monthly, Index 2015=100, non-seasonally adjusted, 1970-01, 2025-09)
# https://fred.stlouisfed.org/series/CCRETT01JPM661N

REXUSDJPY_m_raw = fetch_FRED("CCRETT01JPM661N") 
REXUSDJPY_m_raw = REXUSDJPY_m_raw.rename(columns= 
        {"date": "Time", 
         "CCRETT01JPM661N": "USD-JPY reer CPI-based (Index 2015=100)"
})

REXUSDJPY_m_raw["Time"] = REXUSDJPY_m_raw["Time"].dt.to_period("M").astype(str)

REXUSDJPY_m_raw["Country"] = "JP"
REXUSDJPY_m_raw = REXUSDJPY_m_raw[["Country", "Time", "USD-JPY reer CPI-based (Index 2015=100)"]]

REXUSDJPY_m_raw.tail()

,Country,Time,USD-JPY reer CPI-based (Index 2015=100)
668,JP,2025-09,80.85533
669,JP,2025-10,79.29526
670,JP,2025-11,77.64572
671,JP,2025-12,76.62940
672,JP,2026-01,75.62581


##### **4.2) CONSUMER PRICES (HICP), ALL ITEMS (WHOLE COUNTRY), NSA**

In [164]:
# Consumer prices (HICP), all items (whole country), NSA, Japan (monthly, Index April 2025=111.5, non-seasonally adjusted, 1970-01, 2025-04)
# https://data.ecb.europa.eu/data/datasets/RTD/RTD.M.JP.N.P_C_OV.X

jp_HICP_m_raw = pd.read_csv("Data/HICP_ECB_extracted_raw.csv") 
jp_HICP_m_raw = jp_HICP_m_raw.drop(columns=["DATE"])
jp_HICP_m_raw = jp_HICP_m_raw.rename(columns= 
        {"TIME PERIOD": "Time", 
         "CONSUMER PRICES, ALL ITEMS (WHOLE COUNTRY), NSA (RTD.M.JP.N.P_C_OV.X)": "HICP (NSA)"
})     
                          
jp_HICP_m_raw["Time"] = pd.to_datetime(jp_HICP_m_raw["Time"], format="%Y%b", errors="coerce")
jp_HICP_m_raw["Time"] = jp_HICP_m_raw["Time"].dt.to_period("M").astype(str)

jp_HICP_m_raw["Country"] = "JP"
jp_HICP_m_raw = jp_HICP_m_raw[["Country", "Time", "HICP (NSA)"]]

jp_HICP_m_raw.tail()

,Country,Time,HICP (NSA)
659,JP,2024-12,110.7
660,JP,2025-01,111.2
661,JP,2025-02,110.8
662,JP,2025-03,111.1
663,JP,2025-04,111.5


##### **4.3) JPY-USD SPOT EXCHANGE RATE**

In [165]:
# Japanese Yen to U.S. Dollar Spot Exchange Rate (monthly, non-seasonally adjusted, 1971-01, 2025-09)
# https://fred.stlouisfed.org/series/EXJPUS

JPYUSD_m_raw = fetch_FRED("EXJPUS") 
JPYUSD_m_raw = JPYUSD_m_raw.rename(columns= 
        {"date": "Time", 
         "EXJPUS": "JPY-USD Spot Exchange Rate"
})

JPYUSD_m_raw["Time"] = JPYUSD_m_raw["Time"].dt.to_period("M").astype(str)

JPYUSD_m_raw["Country"] = "JP"
JPYUSD_m_raw = JPYUSD_m_raw[["Country", "Time", "JPY-USD Spot Exchange Rate"]]

JPYUSD_m_raw.tail()

,Country,Time,JPY-USD Spot Exchange Rate
656,JP,2025-09,147.8629
657,JP,2025-10,151.3545
658,JP,2025-11,155.1411
659,JP,2025-12,155.9150
660,JP,2026-01,156.6505


##### **4.4) JPY-USD SPOT EXCHANGE RATE**

In [166]:
# Monthly price and volume of Japan's stock indeces and bond-related instruments (monthly, log, number of securities traded, 2015-01, 2025-10)
# (!!!) Volume column is dangerous, a lot of 0 values, depending on the index, it must be carefully handled 

jp_stock_indices_tickers = {
    "Nikkei 225": "^N225",
    "NYSE Arca Japan Index": "^JPN",
    "iShares 7‑10 Year Japan Government Bond ETF": "236A.T",
    "iShares Core Japan Government Bond ETF": "2561.T"
}

start = "2015-01-01"
end = "2025-10-25"

list_single_country_dfs = []

# We iterate over each country and respective stock index
# We aggregate data through concatenation based on y axis
for index, ticker in jp_stock_indices_tickers.items():
    df = fetch_YFINANCE(ticker, start, end)

# MultiIndex columns
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] for col in df.columns]
    df = df.rename(columns={
        "Close": "Index-specific Closing Price",
        "YearMonth": "Time"
    })
    df["Log Monthly Return"] = np.log(df["Index-specific Closing Price"] / df["Index-specific Closing Price"].shift(1))
    df["Stock Index / Bond-related Instrument"] = index
    df["Stock Index"] = ticker

    df = df[["Stock Index / Bond-related Instrument", "Stock Index", "Time", "Log Monthly Return", "Volume"]]

    list_single_country_dfs.append(df)

jp_stock_m_raw = pd.concat(list_single_country_dfs, ignore_index=True)

jp_stock_m_raw.tail()

,Stock Index / Bond-related Instrument,Stock Index,Time,Log Monthly Return,Volume
328,iShares Core Japan Government Bond ETF,2561.T,2025-06,0.007204,282937
329,iShares Core Japan Government Bond ETF,2561.T,2025-07,-0.016283,276779
330,iShares Core Japan Government Bond ETF,2561.T,2025-08,-0.004130,178697
331,iShares Core Japan Government Bond ETF,2561.T,2025-09,0.003209,273959
332,iShares Core Japan Government Bond ETF,2561.T,2025-10,-0.000916,691174


##### **4.5) TOTAL RESERVES EXCLUDING GOLD (JPN)**

In [167]:
# Total Reserves excluding Gold for Japan (monthly, millions USD, non-seasonally adjusted, 1950-01, 2025-07)
# https://fred.stlouisfed.org/series/TRESEGJPM052N 

jp_reserves_m_raw = fetch_FRED("TRESEGJPM052N") 
jp_reserves_m_raw = jp_reserves_m_raw.rename(columns= 
        {"date": "Time", 
         "TRESEGJPM052N": "Total Treasury Reserves (- Gold)"
})

jp_reserves_m_raw["Time"] = jp_reserves_m_raw["Time"].dt.to_period("M").astype(str)

jp_reserves_m_raw["Country"] = "JP"
jp_reserves_m_raw = jp_reserves_m_raw[["Country", "Time", "Total Treasury Reserves (- Gold)"]]

jp_reserves_m_raw.tail()

,Country,Time,Total Treasury Reserves (- Gold)
897,JP,2025-09,1.237223e+06
898,JP,2025-10,1.238269e+06
899,JP,2025-11,1.245364e+06
900,JP,2025-12,1.252603e+06
901,JP,2026-01,1.259248e+06


##### **4.6) MONETARY AGGREGATES - BROAD MONEY & COMPONENTS - M3 for Japan**

In [168]:
# Monetary Aggregates and Their Components: Broad Money and Components: M3 for Japan (monthly, YPJ , seasonally adjusted, 1980-01, 2023-11)
# https://fred.stlouisfed.org/series/MABMM301JPM189S

jp_m3_m_raw = fetch_FRED("MABMM301JPM189S") 
jp_m3_m_raw = jp_m3_m_raw.rename(columns= 
        {"date": "Time", 
         "MABMM301JPM189S": "Monetary Aggregates - M3 (JPY)"
})

jp_m3_m_raw["Time"] = jp_m3_m_raw["Time"].dt.to_period("M").astype(str)

jp_m3_m_raw["Country"] = "JP"
jp_m3_m_raw = jp_m3_m_raw[["Country", "Time", "Monetary Aggregates - M3 (JPY)"]]

jp_m3_m_raw.tail()

,Country,Time,Monetary Aggregates - M3 (JPY)
522,JP,2023-07,1.588088e+15
523,JP,2023-08,1.590559e+15
524,JP,2023-09,1.592132e+15
525,JP,2023-10,1.595186e+15
526,JP,2023-11,1.597004e+15


##### **4.7) MONETARY AGGREGATES - BROAD MONEY & COMPONENTS - M2 for Japan**

In [169]:
# Monetary Aggregates and Their Components: Broad Money and Components: M2 for Japan (monthly, YPJ, seasonally adjusted, 1955-01, 2017-02)
# https://fred.stlouisfed.org/series/MYAGM2JPM189S

jp_m2_m_raw = fetch_FRED("MYAGM2JPM189S") 
jp_m2_m_raw = jp_m2_m_raw.rename(columns= 
        {"date": "Time", 
         "MYAGM2JPM189S": "Monetary Aggregates - M2 (JPY)"
})

jp_m2_m_raw["Time"] = jp_m2_m_raw["Time"].dt.to_period("M").astype(str)

jp_m2_m_raw["Country"] = "JP"
jp_m2_m_raw = jp_m2_m_raw[["Country", "Time", "Monetary Aggregates - M2 (JPY)"]]

jp_m2_m_raw.tail()

,Country,Time,Monetary Aggregates - M2 (JPY)
741,JP,2016-10,9.492020e+14
742,JP,2016-11,9.530663e+14
743,JP,2016-12,9.560124e+14
744,JP,2017-01,9.591710e+14
745,JP,2017-02,9.632187e+14


##### **4.8) MONETARY AGGREGATES - BROAD MONEY & COMPONENTS - M1 for Japan**

In [170]:
# Monetary Aggregates and Their Components: Broad Money and Components: M1 for Japan (monthly, YPJ, seasonally adjusted, 1960-01, 2023-11)
# https://fred.stlouisfed.org/series/MANMM101JPM189S 

jp_m1_m_raw = fetch_FRED("MANMM101JPM189S") 
jp_m1_m_raw = jp_m1_m_raw.rename(columns= 
        {"date": "Time", 
         "MANMM101JPM189S": "Monetary Aggregates - M1 (JPY)"
})

jp_m1_m_raw["Time"] = jp_m1_m_raw["Time"].dt.to_period("M").astype(str)

jp_m1_m_raw["Country"] = "JP"
jp_m1_m_raw = jp_m1_m_raw[["Country", "Time", "Monetary Aggregates - M1 (JPY)"]]

jp_m1_m_raw.tail()

,Country,Time,Monetary Aggregates - M1 (JPY)
822,JP,2023-07,1.067645e+15
823,JP,2023-08,1.072350e+15
824,JP,2023-09,1.076164e+15
825,JP,2023-10,1.079579e+15
826,JP,2023-11,1.081546e+15


##### **4.9) TOTAL CREDIT TO PRIVATE NON-FINANCIAL SECTOR**

In [171]:
# Total Credit to Private Non-Financial Sector, Adjusted for Breaks, for Japan (quarterly, percentage of GDP, breaks adjusted, 1964-01, 2025-01)
# https://fred.stlouisfed.org/series/QJPPAM770A

jp_credit_pnf_m_raw = fetch_FRED("QJPPAM770A") 
jp_credit_pnf_m_raw = jp_credit_pnf_m_raw.rename(columns= 
        {"date": "Time", 
         "QJPPAM770A": "Total Credit - Private Non-Financial (%GDP)"
})
jp_credit_pnf_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_credit_pnf_m_raw["Time"] = pd.PeriodIndex(jp_credit_pnf_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_credit_pnf_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Total Credit - Private Non-Financial (%GDP)": row["Total Credit - Private Non-Financial (%GDP)"] 
        })

jp_credit_pnf_m_raw = pd.DataFrame(expanded_rows)
jp_credit_pnf_m_raw["Country"] = "JP"
jp_credit_pnf_m_raw = jp_credit_pnf_m_raw[["Country", "Time", "Total Credit - Private Non-Financial (%GDP)"]]

jp_credit_pnf_m_raw.tail()

,Country,Time,Total Credit - Private Non-Financial (%GDP)
724,JP,2025-02,180.1
725,JP,2025-03,180.1
726,JP,2025-04,179.3
727,JP,2025-05,179.3
728,JP,2025-06,179.3


##### **4.10) TOTAL CREDIT TO GENERAL GOVERNMENT**

In [172]:
# Total Credit to General Government, Adjusted for Breaks, for Japan (quarterly, percentage of GDP, breaks adjusted, 1997-01, 2025-01)
# https://fred.stlouisfed.org/series/QJPGAM770A

jp_credit_gg_m_raw = fetch_FRED("QJPGAM770A") 
jp_credit_gg_m_raw = jp_credit_gg_m_raw.rename(columns= 
        {"date": "Time", 
         "QJPGAM770A": "Total Credit - General Government (%GDP)"
})
jp_credit_gg_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_credit_gg_m_raw["Time"] = pd.PeriodIndex(jp_credit_gg_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_credit_gg_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Total Credit - General Government (%GDP)": row["Total Credit - General Government (%GDP)"] 
        })

jp_credit_gg_m_raw = pd.DataFrame(expanded_rows)
jp_credit_gg_m_raw["Country"] = "JP"
jp_credit_gg_m_raw = jp_credit_gg_m_raw[["Country", "Time", "Total Credit - General Government (%GDP)"]]

jp_credit_gg_m_raw.tail()

,Country,Time,Total Credit - General Government (%GDP)
328,JP,2025-02,200.6
329,JP,2025-03,200.6
330,JP,2025-04,198.1
331,JP,2025-05,198.1
332,JP,2025-06,198.1


##### **4.11) TOTAL CREDIT TO HOUSEHOLDS & NPISHs**

In [173]:
# Total Credit to households and NPISHs, Adjusted for Breaks, for Japan (quarterly, percentage of GDP, breaks adjusted, 1964-01, 2025-01)
# https://fred.stlouisfed.org/series/QJPHAM770A

jp_credit_households_m_raw = fetch_FRED("QJPHAM770A") 
jp_credit_households_m_raw = jp_credit_households_m_raw.rename(columns= 
        {"date": "Time", 
         "QJPHAM770A": "Total Credit - Households & NPISHs (%GDP)"
})
jp_credit_households_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_credit_households_m_raw["Time"] = pd.PeriodIndex(jp_credit_households_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_credit_households_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Total Credit - Households & NPISHs (%GDP)": row["Total Credit - Households & NPISHs (%GDP)"] 
        })

jp_credit_households_m_raw = pd.DataFrame(expanded_rows)
jp_credit_households_m_raw["Country"] = "JP"
jp_credit_households_m_raw = jp_credit_households_m_raw[["Country", "Time", "Total Credit - Households & NPISHs (%GDP)"]]

jp_credit_households_m_raw.tail()

,Country,Time,Total Credit - Households & NPISHs (%GDP)
724,JP,2025-02,64.5
725,JP,2025-03,64.5
726,JP,2025-04,64.0
727,JP,2025-05,64.0
728,JP,2025-06,64.0


##### **4.12) REAL GDP**

In [174]:
# Real Gross Domestic Product for Japan (quarterly, billions of chained 2015 JPY, seasonally adjusted, 1994-01, 2025-04)
# https://fred.stlouisfed.org/series/JPNRGDPEXP

jp_rgdp_m_raw = fetch_FRED("JPNRGDPEXP") 
jp_rgdp_m_raw = jp_rgdp_m_raw.rename(columns= 
        {"date": "Time", 
         "JPNRGDPEXP": "Real GDP (billions chained 2015 JPY)"
})
jp_rgdp_m_raw["Country"] = "JP"

# Increasing data granularity from quarterly to monthly data by extending the quarter value to single months 
jp_rgdp_m_raw["Time"] = pd.PeriodIndex(jp_rgdp_m_raw["Time"], freq="Q").to_timestamp()
expanded_rows = []

for _, row in jp_rgdp_m_raw.iterrows():
    quarter_end = row["Time"]
    start_month = quarter_end - pd.offsets.QuarterEnd(startingMonth=3) + pd.DateOffset(days=1)
    for i in range(3):
        month = (start_month + pd.DateOffset(months=i)).strftime("%Y-%m")
        expanded_rows.append({
            "Country": row["Country"],
            "Time": month,
            "Real GDP (billions chained 2015 JPY)": row["Real GDP (billions chained 2015 JPY)"] 
        })

jp_rgdp_m_raw = pd.DataFrame(expanded_rows)
jp_rgdp_m_raw["Country"] = "JP"
jp_rgdp_m_raw = jp_rgdp_m_raw[["Country", "Time", "Real GDP (billions chained 2015 JPY)"]]

jp_rgdp_m_raw.tail()

,Country,Time,Real GDP (billions chained 2015 JPY)
379,JP,2025-08,589417.7
380,JP,2025-09,589417.7
381,JP,2025-10,589727.6
382,JP,2025-11,589727.6
383,JP,2025-12,589727.6


##### **4.13) LONG-TERM GOVERNMENT BOND YIELDS - 10-YEAR**

In [175]:
# Interest Rates: Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Japan (monthly, percent, non-seasonally adjusted, 1989-01, 2025-09)
# https://fred.stlouisfed.org/series/IRLTLT01JPM156N

jp_10ygb_m_raw = fetch_FRED("IRLTLT01JPM156N") 
jp_10ygb_m_raw = jp_10ygb_m_raw.rename(columns= 
        {"date": "Time", 
         "IRLTLT01JPM156N": "10-Year Gov Bond Yields (%)"
})

jp_10ygb_m_raw["Time"] = jp_10ygb_m_raw["Time"].dt.to_period("M").astype(str)

jp_10ygb_m_raw["Country"] = "JP"
jp_10ygb_m_raw = jp_10ygb_m_raw[["Country", "Time", "10-Year Gov Bond Yields (%)"]]

jp_10ygb_m_raw.tail()

,Country,Time,10-Year Gov Bond Yields (%)
440,JP,2025-09,1.645
441,JP,2025-10,1.655
442,JP,2025-11,1.805
443,JP,2025-12,2.060
444,JP,2026-01,2.240


##### **4.14) IMMEDIATE RATES (< 24 Hours): CALL MONEY/INTERBANK RATE**

In [176]:
# Interest Rates: Immediate Rates (< 24 Hours): Call Money/Interbank Rate: Total for Japan (monthly, percent, non-seasonally adjusted, 1985-01, 2025-09)
# https://fred.stlouisfed.org/series/IRSTCI01JPM156N

jp_cmibr_m_raw = fetch_FRED("IRSTCI01JPM156N") 
jp_cmibr_m_raw = jp_cmibr_m_raw.rename(columns= 
        {"date": "Time", 
         "IRSTCI01JPM156N": "Call Money/Interbank Immediate (%)"
})

jp_cmibr_m_raw["Time"] = jp_cmibr_m_raw["Time"].dt.to_period("M").astype(str)

jp_cmibr_m_raw["Country"] = "JP"
jp_cmibr_m_raw = jp_cmibr_m_raw[["Country", "Time", "Call Money/Interbank Immediate (%)"]]

jp_cmibr_m_raw.tail()

,Country,Time,Call Money/Interbank Immediate (%)
482,JP,2025-09,0.477
483,JP,2025-10,0.477
484,JP,2025-11,0.478
485,JP,2025-12,0.557
486,JP,2026-01,0.728


##### **4.15) NATURAL RATE OF INTEREST: SHORT-TERM (1-year) & LONG-TERM (10-year)**

In [177]:
# Natural rate of interest: Short-term (1-year) and long-term (10-year) natural rate of interest (mean estimates and 95% confidence intervals, total for Japan (quarterly, percent, non-seasonally adjusted, 1995-01, 2025-12))
# https://sites.google.com/site/jnakajimaweb/rstar
# Nakajima, J., N. Sudo, Y. Hogen, and Y. Takizuka (2023). "On the estimation of the natural yield curve" Discussion Paper Series A.753, Institute of Economic Research, Hitotsubashi University
# (!!!) Quarterly data, 1Y_Mean and 10Y_Mean

jp_nir_m_raw = pd.read_csv("Data/nakajima et al. (2023).csv")

# (!!!) The dataset is built missing column headlines, so we have to restructure it 
# (!!!) We take only the average values
jp_nir_m_raw = jp_nir_m_raw.iloc[1:].reset_index(drop=True) 
jp_nir_m_raw = jp_nir_m_raw.rename(columns={
    jp_nir_m_raw.columns[0]: "YYYYQ",
    jp_nir_m_raw.columns[1]: "1Y_Mean",
    jp_nir_m_raw.columns[4]: "10Y_Mean"
})

# We need to isolate year and quarter 
jp_nir_m_raw["Year"] = jp_nir_m_raw["YYYYQ"].astype(str).str[:4].astype(int)
jp_nir_m_raw["Quarter"] = jp_nir_m_raw["YYYYQ"].astype(str).str[-1].astype(int)
# Quarter start month
quarter_start_month = {1: 1, 2: 4, 3: 7, 4: 10}
jp_nir_m_raw["StartMonth"] = jp_nir_m_raw["Quarter"].map(quarter_start_month)

# Quarter start date
jp_nir_m_raw["QuarterStart"] = pd.to_datetime(
    dict(year=jp_nir_m_raw["Year"], month=jp_nir_m_raw["StartMonth"], day=1),
    errors="coerce"
)

# Expansion of quarterly data to monthly (flat within quarter)
jp_nir_m_raw = (
    jp_nir_m_raw
    .loc[jp_nir_m_raw.index.repeat(3)]
    .assign(
        Time=lambda x: x["QuarterStart"] + x.groupby(level=0).cumcount().map(lambda i: pd.DateOffset(months=i))
    )
)
jp_nir_m_raw["Time"] = pd.to_datetime(jp_nir_m_raw["Time"], errors="coerce")
jp_nir_m_raw = (
    jp_nir_m_raw
    .assign(
        Time=lambda x: x["Time"].dt.to_period("M").astype(str),
        Country="JP"
    )
    [["Country", "Time", "1Y_Mean", "10Y_Mean"]]
    .reset_index(drop=True)
)
jp_nir_m_raw = jp_nir_m_raw.rename(columns={
    "1Y_Mean": "Est. 1-year Neutral Interest Rate (%)",
    "10Y_Mean": "Est. 10-year Neutral Interest Rate (%)"
})

jp_nir_m_raw.tail()

,Country,Time,Est. 1-year Neutral Interest Rate (%),Est. 10-year Neutral Interest Rate (%)
367,JP,2025-08,-0.27,0.481
368,JP,2025-09,-0.27,0.481
369,JP,2025-10,-0.258,0.497
370,JP,2025-11,-0.258,0.497
371,JP,2025-12,-0.258,0.497


##### **4.16) BANK OF JAPAN'S TOTAL ASSET FROM BALANCE SHEET**

In [178]:
# Bank of Japan’s total assets from its official balance sheet (monthly, 100 Million Yen, non-seasonally adjusted, 1998-04, 2025-12)
# https://fred.stlouisfed.org/series/JPNASSETS

jp_bojta_m_raw = fetch_FRED("JPNASSETS") 
jp_bojta_m_raw = jp_bojta_m_raw.rename(columns= 
        {"date": "Time", 
         "JPNASSETS": "BoJ’s Total Assets (100 Million Yen)"
})

jp_bojta_m_raw["Time"] = jp_bojta_m_raw["Time"].dt.to_period("M").astype(str)

jp_bojta_m_raw["Country"] = "JP"
jp_bojta_m_raw = jp_bojta_m_raw[["Country", "Time", "BoJ’s Total Assets (100 Million Yen)"]]

jp_bojta_m_raw.tail()

,Country,Time,BoJ’s Total Assets (100 Million Yen)
329,JP,2025-09,6957819.0
330,JP,2025-10,6963487.0
331,JP,2025-11,6979490.0
332,JP,2025-12,6777762.0
333,JP,2026-01,6828680.0


##### **4.17) MARKET YIELD ON U.S. TREASURY SECURITY AT 10-Year CONSTANT MATURITY**

In [179]:
# Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis (daily, percent, non-seasonally adjusted, 1962-01, 2026-01)
# https://fred.stlouisfed.org/series/DGS10

jp_ust_m_raw = fetch_FRED("DGS10") 
jp_ust_m_raw = jp_ust_m_raw.rename(columns= 
        {"date": "Time", 
         "DGS10": "10-Year US T-Bills Yield (%)"
})
jp_ust_m_raw["Time"] = pd.to_datetime(jp_ust_m_raw["Time"])
jp_ust_m_raw = (
    jp_ust_m_raw
    .set_index("Time")
    .resample("M")
    .mean()
    .reset_index()
)
jp_ust_m_raw["Time"] = jp_ust_m_raw["Time"].dt.to_period("M").astype(str)

jp_ust_m_raw["Country"] = "JP"
jp_ust_m_raw = jp_ust_m_raw[["Country", "Time", "10-Year US T-Bills Yield (%)"]]

jp_ust_m_raw.tail()

,Country,Time,10-Year US T-Bills Yield (%)
765,JP,2025-10,4.061818
766,JP,2025-11,4.093889
767,JP,2025-12,4.143182
768,JP,2026-01,4.213500
769,JP,2026-02,4.141176


##### **4.18) CBOE VOLATILITY INDEX: VIX**

In [180]:
# CBOE Volatility Index: VIX (daily, index, non-seasonally adjusted, 1990-01, 2026-01)
# https://fred.stlouisfed.org/series/VIXCLS

jp_vix_m_raw = fetch_FRED("VIXCLS") 
jp_vix_m_raw = jp_vix_m_raw.rename(columns= 
        {"date": "Time", 
         "VIXCLS": "CBOE-VIX"
})
jp_vix_m_raw["Time"] = pd.to_datetime(jp_vix_m_raw["Time"])
jp_vix_m_raw = (
    jp_vix_m_raw
    .set_index("Time")
    .resample("M")
    .mean()
    .reset_index()
)
jp_vix_m_raw["Time"] = jp_vix_m_raw["Time"].dt.to_period("M").astype(str)

jp_vix_m_raw["Country"] = "JP"
jp_vix_m_raw = jp_vix_m_raw[["Country", "Time", "CBOE-VIX"]]

jp_vix_m_raw.tail()

,Country,Time,CBOE-VIX
429,JP,2025-10,18.086522
430,JP,2025-11,19.769500
431,JP,2025-12,15.548182
432,JP,2026-01,16.179048
433,JP,2026-02,19.172632


##### **4.19) CENTRAL GOVERNMENT DEBT (% of GDP)**

In [181]:
# Central government debt, total (% of GDP) for Japan (annual, % of GDP, non-seasonally adjusted, 1990, 2022)
# https://fred.stlouisfed.org/series/DEBTTLJPA188A 

jp_cgdebt_m_raw = fetch_FRED("DEBTTLJPA188A")
jp_cgdebt_m_raw = jp_cgdebt_m_raw.rename(columns={
    "date": "Time",
    "DEBTTLJPA188A": "Central Government Debt (% GDP)"
})
jp_cgdebt_m_raw["Time"] = pd.to_datetime(jp_cgdebt_m_raw["Time"])
jp_cgdebt_m_raw = (
    jp_cgdebt_m_raw
    .set_index("Time")
    .resample("MS")        
    .ffill()              
    .reset_index()
)
jp_cgdebt_m_raw["Time"] = jp_cgdebt_m_raw["Time"].dt.to_period("M").astype(str)
jp_cgdebt_m_raw["Country"] = "JP"
jp_cgdebt_m_raw = jp_cgdebt_m_raw[
    ["Country", "Time", "Central Government Debt (% GDP)"]
]

jp_cgdebt_m_raw.tail()

,Country,Time,Central Government Debt (% GDP)
380,JP,2021-09,216.135005
381,JP,2021-10,216.135005
382,JP,2021-11,216.135005
383,JP,2021-12,216.135005
384,JP,2022-01,215.906360


##### **4.20) OUTSTANDING DOMESTIC PRIVATE DEBT SECURITIES TO GDP**

In [182]:
# Outstanding Domestic Private Debt Securities to GDP for Japan (annual, % of GDP, non-seasonally adjusted, 1997, 2020)
# https://fred.stlouisfed.org/series/DDDM03JPA156NWDB 

jp_dpdebt_m_raw = fetch_FRED("DDDM03JPA156NWDB")
jp_dpdebt_m_raw = jp_dpdebt_m_raw.rename(columns={
    "date": "Time",
    "DDDM03JPA156NWDB": "Domestic Private Debt Securities (% GDP)"
})
jp_dpdebt_m_raw["Time"] = pd.to_datetime(jp_dpdebt_m_raw["Time"])
jp_dpdebt_m_raw = (
    jp_dpdebt_m_raw
    .set_index("Time")
    .resample("MS")        
    .ffill()              
    .reset_index()
)
jp_dpdebt_m_raw["Time"] = jp_dpdebt_m_raw["Time"].dt.to_period("M").astype(str)
jp_dpdebt_m_raw["Country"] = "JP"
jp_dpdebt_m_raw = jp_dpdebt_m_raw[
    ["Country", "Time", "Domestic Private Debt Securities (% GDP)"]
]

jp_dpdebt_m_raw.tail()

,Country,Time,Domestic Private Debt Securities (% GDP)
356,JP,2019-09,56.49202
357,JP,2019-10,56.49202
358,JP,2019-11,56.49202
359,JP,2019-12,56.49202
360,JP,2020-01,68.73470


##### **4.21) OUTSTANDING DOMESTIC PUBLIC DEBT SECURITIES TO GDP**

In [183]:
# Outstanding Domestic Public Debt Securities to GDP for Japan (annual, % of GDP, non-seasonally adjusted, 1997, 2020)
# https://fred.stlouisfed.org/series/DDDM04JPA156NWDB  

jp_dudebt_m_raw = fetch_FRED("DDDM04JPA156NWDB")
jp_dudebt_m_raw = jp_dudebt_m_raw.rename(columns={
    "date": "Time",
    "DDDM04JPA156NWDB": "Domestic Public Debt Securities (% GDP)"
})
jp_dudebt_m_raw["Time"] = pd.to_datetime(jp_dudebt_m_raw["Time"])
jp_dudebt_m_raw = (
    jp_dudebt_m_raw
    .set_index("Time")
    .resample("MS")        
    .ffill()              
    .reset_index()
)
jp_dudebt_m_raw["Time"] = jp_dudebt_m_raw["Time"].dt.to_period("M").astype(str)
jp_dudebt_m_raw["Country"] = "JP"
jp_dudebt_m_raw = jp_dudebt_m_raw[
    ["Country", "Time", "Domestic Public Debt Securities (% GDP)"]
]

jp_dudebt_m_raw.tail()

,Country,Time,Domestic Public Debt Securities (% GDP)
356,JP,2019-09,184.320
357,JP,2019-10,184.320
358,JP,2019-11,184.320
359,JP,2019-12,184.320
360,JP,2020-01,217.026


##### **4.22) AVERAGE INTEREST RATES ON TIME DEPOSITS BY TERM**

In [184]:
# Average Interest Rates on Certificates of Deposit 60 Days - 89 Days for Japan (monthly, %, non-seasonally adjusted, 1985/06, 2022/03)
# Time Deposits/ 10 Million Yen or More/3 Months for Japan (monthly, %, non-seasonally adjusted, 2022/03, 2026/02)
# https://www.boj.or.jp/en/statistics/dl/depo/tentorate/index.htm (IR02STRDCDNITL03)
# https://www.boj.or.jp/en/statistics/dl/depo/tentorate/index.htm (IR02DLDR43TL43MN)

# BoJ df until 2022/03
jp_depi1_m_raw = pd.read_csv("Data/BoJ_deposits_i_IR02STRDCDNITL03.csv")
jp_depi1_m_raw = jp_depi1_m_raw.rename(columns={
    "Average Interest Rates on Certificates of Deposit <New Issues>(City Banks, etc.)/60 Days - 89 Days (IR02'STRDCDNITL03)": "Deposit Interest Rate (%)"
})
jp_depi1_m_raw["Time"] = pd.to_datetime(jp_depi1_m_raw["Time"])
jp_depi1_m_raw["Time"] = jp_depi1_m_raw["Time"].dt.to_period("M").astype(str)
jp_depi1_m_raw = jp_depi1_m_raw[
    ["Time", "Deposit Interest Rate (%)"]
]
jp_depi1_m_raw = jp_depi1_m_raw.dropna()

# BoJ df from 2022/03 onwards
jp_depi2_m_raw = pd.read_csv("Data/BoJ_deposits_i_extended_IR02DLDR43TL43MN.csv")
jp_depi2_m_raw = jp_depi2_m_raw.rename(columns={
    "Time Deposits/ 10 Million Yen or More/3 Months (IR02'DLDR43TL43MN)": "Deposit Interest Rate (%)"
})
jp_depi2_m_raw["Time"] = pd.to_datetime(jp_depi2_m_raw["Time"])
jp_depi2_m_raw["Time"] = jp_depi2_m_raw["Time"].dt.to_period("M").astype(str)
jp_depi2_m_raw = jp_depi2_m_raw[
    ["Time", "Deposit Interest Rate (%)"]
]
jp_depi2_m_raw = jp_depi2_m_raw.dropna()

# Merging
jp_depi_m_raw = pd.merge(jp_depi1_m_raw, jp_depi2_m_raw, on=["Time", "Deposit Interest Rate (%)"], how="outer")
jp_depi_m_raw["Country"] = "JP"
jp_depi_m_raw = jp_depi_m_raw[
    ["Country", "Time", "Deposit Interest Rate (%)"]
]

jp_depi_m_raw.tail()

,Country,Time,Deposit Interest Rate (%)
484,JP,2025-10,0.225
485,JP,2025-11,0.225
486,JP,2025-12,0.225
487,JP,2026-01,0.227
488,JP,2026-02,0.296


##### **4.23) AVERAGE CONTRACT INTEREST RATES ON LOANS AND DISCOUNTS**

In [185]:
# Average Interest Rates on New Loans and Discounts, Total, Domestically Licensed Banks for Japan (monthly, %, non-seasonally adjusted, 1993/10, 2025/12)
# https://www.boj.or.jp/en/statistics/dl/loan/yaku/index.htm (IR04DLLR2CIDBNL1)

jp_loai_m_raw = pd.read_csv("Data/BoJ_loans_i_IR04DLLR2CIDBNL1.csv")
jp_loai_m_raw = jp_loai_m_raw.rename(columns={
    "New Loans and Discounts/Total/Domestically Licensed Banks (IR04'DLLR2CIDBNL1)": "Loan Interest Rate (%)"
})
jp_loai_m_raw["Time"] = pd.to_datetime(jp_loai_m_raw["Time"])
jp_loai_m_raw["Time"] = jp_loai_m_raw["Time"].dt.to_period("M").astype(str)
jp_loai_m_raw["Country"] = "JP"
jp_loai_m_raw = jp_loai_m_raw[
    ["Country", "Time", "Loan Interest Rate (%)"]
]
jp_loai_m_raw = jp_loai_m_raw.dropna()

jp_loai_m_raw.tail()

,Country,Time,Loan Interest Rate (%)
787,JP,2025-08,1.149
788,JP,2025-09,1.415
789,JP,2025-10,1.272
790,JP,2025-11,1.139
791,JP,2025-12,1.404


##### **4.24) BANKING SECTOR DEFAULT PROXY (JAPAN)**

In [186]:
# NEXT FUNDS TOPIX Banks Exchange Traded Fund, 1615.T, (monthly, %, non-seasonally adjusted, 1993/10, 2025/12)
# https://finance.yahoo.com/quote/1615.T/ (1615.T)

jp_def_m_raw = fetch_YFINANCE("1615.T", "2000-01-01", "2026-02-25")
jp_def_m_raw.columns = ["Time", "1615.T-Price", "1615.T-Volume"]

jp_def_m_raw["Time"] = jp_def_m_raw["Time"].dt.to_timestamp()
jp_def_m_raw["Time"] = jp_def_m_raw["Time"].dt.to_period("M").astype(str)
jp_def_m_raw["Country"] = "JP"
jp_def_m_raw = jp_def_m_raw[
    ["Country", "Time", "1615.T-Price"]
]
jp_def_m_raw = jp_def_m_raw.dropna()

jp_def_m_raw.tail()

,Country,Time,1615.T-Price
213,JP,2025-10,468.299988
214,JP,2025-11,505.500000
215,JP,2025-12,528.200012
216,JP,2026-01,598.299988
217,JP,2026-02,647.900024


### **5) DATA MERGING**

In [187]:
# Broad Money Dataframe
jp_broad_money_df = pd.merge(jp_m1_m_raw, jp_m2_m_raw, on=["Country", "Time"], how="outer")
jp_broad_money_df = pd.merge(jp_broad_money_df, jp_m3_m_raw, on=["Country", "Time"], how="outer")

jp_broad_money_df.to_csv("Data/Aggregated/jp_broad_money_df.csv", index=False)
jp_broad_money_df.tail()

,Country,Time,Monetary Aggregates - M1 (JPY),Monetary Aggregates - M2 (JPY),Monetary Aggregates - M3 (JPY)
822,JP,2023-07,1.067645e+15,NaN,1.588088e+15
823,JP,2023-08,1.072350e+15,NaN,1.590559e+15
824,JP,2023-09,1.076164e+15,NaN,1.592132e+15
825,JP,2023-10,1.079579e+15,NaN,1.595186e+15
826,JP,2023-11,1.081546e+15,NaN,1.597004e+15


In [188]:
# Credit Demand Dataframe
jp_credit_demand_df = pd.merge(jp_credit_pnf_m_raw, jp_credit_gg_m_raw, on=["Country", "Time"], how="outer")
jp_credit_demand_df = pd.merge(jp_credit_demand_df, jp_credit_households_m_raw , on=["Country", "Time"], how="outer")

jp_credit_demand_df.to_csv("Data/Aggregated/jp_credit_demand_df.csv", index=False)
jp_credit_demand_df.tail()

,Country,Time,Total Credit - Private Non-Financial (%GDP),Total Credit - General Government (%GDP),Total Credit - Households & NPISHs (%GDP)
724,JP,2025-02,180.1,200.6,64.5
725,JP,2025-03,180.1,200.6,64.5
726,JP,2025-04,179.3,198.1,64.0
727,JP,2025-05,179.3,198.1,64.0
728,JP,2025-06,179.3,198.1,64.0


In [189]:
# Bank Reserves Dataframe
jp_bank_reserves_df = jp_reserves_m_raw.copy()

jp_bank_reserves_df.to_csv("Data/Aggregated/jp_bank_reserves_df.csv", index=False)
jp_bank_reserves_df.tail()

,Country,Time,Total Treasury Reserves (- Gold)
897,JP,2025-09,1.237223e+06
898,JP,2025-10,1.238269e+06
899,JP,2025-11,1.245364e+06
900,JP,2025-12,1.252603e+06
901,JP,2026-01,1.259248e+06


In [190]:
# BoJ's Total Assets
jp_bojta_df = jp_bojta_m_raw.copy()

jp_bojta_df.to_csv("Data/Aggregated/jp_boj_total_assets_df.csv", index=False)
jp_bojta_df.tail()

,Country,Time,BoJ’s Total Assets (100 Million Yen)
329,JP,2025-09,6957819.0
330,JP,2025-10,6963487.0
331,JP,2025-11,6979490.0
332,JP,2025-12,6777762.0
333,JP,2026-01,6828680.0


In [191]:
# Policy Rate Dataframe
jp_policy_rate_df = pd.merge(jp_10ygb_m_raw, jp_cmibr_m_raw, on=["Country", "Time"], how="outer")
jp_policy_rate_df = pd.merge(jp_policy_rate_df, jp_nir_m_raw, on=["Country", "Time"], how="outer")

jp_policy_rate_df.to_csv("Data/Aggregated/jp_policy_rate_df.csv", index=False)
jp_policy_rate_df.tail()

,Country,Time,10-Year Gov Bond Yields (%),Call Money/Interbank Immediate (%),Est. 1-year Neutral Interest Rate (%),Est. 10-year Neutral Interest Rate (%)
482,JP,2025-09,1.645,0.477,-0.27,0.481
483,JP,2025-10,1.655,0.477,-0.258,0.497
484,JP,2025-11,1.805,0.478,-0.258,0.497
485,JP,2025-12,2.060,0.557,-0.258,0.497
486,JP,2026-01,2.240,0.728,NaN,NaN


In [192]:
# Exchange Rate Dataframe
jp_exchange_rate_df = pd.merge(REXUSDJPY_m_raw, JPYUSD_m_raw, on=["Country", "Time"], how="outer")

jp_exchange_rate_df.to_csv("Data/Aggregated/jp_exchange_rate_df.csv", index=False)
jp_exchange_rate_df.tail()

,Country,Time,USD-JPY reer CPI-based (Index 2015=100),JPY-USD Spot Exchange Rate
668,JP,2025-09,80.85533,147.8629
669,JP,2025-10,79.29526,151.3545
670,JP,2025-11,77.64572,155.1411
671,JP,2025-12,76.62940,155.9150
672,JP,2026-01,75.62581,156.6505


In [193]:
# Inflation Dataframe
jp_inflation_df = jp_HICP_m_raw.copy()

jp_inflation_df.to_csv("Data/Aggregated/jp_inflation_df.csv", index=False)
jp_inflation_df.tail()

,Country,Time,HICP (NSA)
659,JP,2024-12,110.7
660,JP,2025-01,111.2
661,JP,2025-02,110.8
662,JP,2025-03,111.1
663,JP,2025-04,111.5


In [194]:
# Real GDP Dataframe
jp_rgdp_df = jp_rgdp_m_raw.copy()

jp_rgdp_df.to_csv("Data/Aggregated/jp_rgdp_df.csv", index=False)
jp_rgdp_df.tail()

,Country,Time,Real GDP (billions chained 2015 JPY)
379,JP,2025-08,589417.7
380,JP,2025-09,589417.7
381,JP,2025-10,589727.6
382,JP,2025-11,589727.6
383,JP,2025-12,589727.6


In [195]:
# Stock Indeces Dataframe
jp_stocks_pv_df = jp_stock_m_raw.copy()

jp_stocks_pv_df.to_csv("Data/Aggregated/jp_stocks_pv_df.csv", index=False)
jp_stocks_pv_df.tail()

,Stock Index / Bond-related Instrument,Stock Index,Time,Log Monthly Return,Volume
328,iShares Core Japan Government Bond ETF,2561.T,2025-06,0.007204,282937
329,iShares Core Japan Government Bond ETF,2561.T,2025-07,-0.016283,276779
330,iShares Core Japan Government Bond ETF,2561.T,2025-08,-0.004130,178697
331,iShares Core Japan Government Bond ETF,2561.T,2025-09,0.003209,273959
332,iShares Core Japan Government Bond ETF,2561.T,2025-10,-0.000916,691174


In [196]:
# Debt Level Dataframe

jp_debt_df = pd.merge(jp_cgdebt_m_raw, jp_dpdebt_m_raw, on=["Country", "Time"], how="outer")
jp_debt_df = pd.merge(jp_debt_df, jp_dudebt_m_raw, on=["Country", "Time"], how="outer")

jp_debt_df.to_csv("Data/Aggregated/jp_debt_df.csv", index=False)
jp_debt_df.tail()

,Country,Time,Central Government Debt (% GDP),Domestic Private Debt Securities (% GDP),Domestic Public Debt Securities (% GDP)
380,JP,2021-09,216.135005,NaN,NaN
381,JP,2021-10,216.135005,NaN,NaN
382,JP,2021-11,216.135005,NaN,NaN
383,JP,2021-12,216.135005,NaN,NaN
384,JP,2022-01,215.906360,NaN,NaN


In [197]:
# Net Interest Margin Dataframe
jp_nim_df = pd.merge(jp_loai_m_raw, jp_depi_m_raw, on=["Country", "Time"], how="outer")

jp_nim_df.to_csv("Data/Aggregated/jp_nim_df.csv", index=False)
jp_nim_df.tail()

,Country,Time,Loan Interest Rate (%),Deposit Interest Rate (%)
484,JP,2025-10,1.272,0.225
485,JP,2025-11,1.139,0.225
486,JP,2025-12,1.404,0.225
487,JP,2026-01,NaN,0.227
488,JP,2026-02,NaN,0.296


In [198]:
# Default Rate Proxy Dataframe 
jp_stress_df = jp_def_m_raw.copy()

jp_stress_df.to_csv("Data/Aggregated/jp_stress_df.csv", index=False)
jp_stress_df.tail()

,Country,Time,1615.T-Price
213,JP,2025-10,468.299988
214,JP,2025-11,505.500000
215,JP,2025-12,528.200012
216,JP,2026-01,598.299988
217,JP,2026-02,647.900024


In [199]:
# Control Dataframe
jp_control_df = pd.merge(jp_ust_m_raw, jp_vix_m_raw, on=["Country", "Time"], how="outer")

jp_control_df.to_csv("Data/Aggregated/jp_control_df.csv", index=False)
jp_control_df.tail()

,Country,Time,10-Year US T-Bills Yield (%),CBOE-VIX
765,JP,2025-10,4.061818,18.086522
766,JP,2025-11,4.093889,19.769500
767,JP,2025-12,4.143182,15.548182
768,JP,2026-01,4.213500,16.179048
769,JP,2026-02,4.141176,19.172632


In [200]:
# Aggregated JP Dataframe
jp_aggregated_df = pd.merge(jp_broad_money_df, jp_credit_demand_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_bank_reserves_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_policy_rate_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_exchange_rate_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_inflation_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_rgdp_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_debt_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_bojta_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_nim_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_stress_df, on=["Country", "Time"], how="outer")
jp_aggregated_df = pd.merge(jp_aggregated_df, jp_control_df, on=["Country", "Time"], how="outer")

jp_aggregated_df.to_csv("Data/Aggregated/jp_aggregated_df.csv", index=False)
jp_aggregated_df.tail()

,Country,Time,Monetary Aggregates - M1 (JPY),Monetary Aggregates - M2 (JPY),Monetary Aggregates - M3 (JPY),Total Credit - Private Non-Financial (%GDP),Total Credit - General Government (%GDP),Total Credit - Households & NPISHs (%GDP),Total Treasury Reserves (- Gold),10-Year Gov Bond Yields (%),...,Real GDP (billions chained 2015 JPY),Central Government Debt (% GDP),Domestic Private Debt Securities (% GDP),Domestic Public Debt Securities (% GDP),BoJ’s Total Assets (100 Million Yen),Loan Interest Rate (%),Deposit Interest Rate (%),1615.T-Price,10-Year US T-Bills Yield (%),CBOE-VIX
898,JP,2025-10,NaN,NaN,NaN,NaN,NaN,NaN,1.238269e+06,1.655,...,589727.6,NaN,NaN,NaN,6963487.0,1.272,0.225,468.299988,4.061818,18.086522
899,JP,2025-11,NaN,NaN,NaN,NaN,NaN,NaN,1.245364e+06,1.805,...,589727.6,NaN,NaN,NaN,6979490.0,1.139,0.225,505.500000,4.093889,19.769500
900,JP,2025-12,NaN,NaN,NaN,NaN,NaN,NaN,1.252603e+06,2.060,...,589727.6,NaN,NaN,NaN,6777762.0,1.404,0.225,528.200012,4.143182,15.548182
901,JP,2026-01,NaN,NaN,NaN,NaN,NaN,NaN,1.259248e+06,2.240,...,NaN,NaN,NaN,NaN,6828680.0,NaN,0.227,598.299988,4.213500,16.179048
902,JP,2026-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.296,647.900024,4.141176,19.172632
